In [113]:
from collections import Counter, defaultdict
import json
import re

In [180]:
with open("banner/courses.json", "r") as f:
    sections = json.load(f)

with open("calendar/degrees_data.json", "r") as f:
    degrees = json.load(f)

with open("calendar/course_data.json", "r") as f:
    courses= json.load(f)

with open("map/buildings.json", "r") as f:
    buildings = json.load(f)

In [191]:
cleaned_sections = dict()
cleaned_course_sections = defaultdict(list)
programs = defaultdict(list)
rooms = defaultdict(set)

for sec in sections.values():
    
    clean_sec = {
        'term': sec['term'],
        'crn': sec['courseReferenceNumber'],
        'subject_code': sec['subject'],
        'subject': sec['subjectDescription'],
        'course_number': sec['courseNumber'],
        'course_title': sec['courseTitle'],
        'sequence': sec['sequenceNumber'],
        'type': sec['scheduleTypeDescription'],
        'is_linked': sec['isSectionLinked'],
        'credit': sec['creditHours'],
        'faculty': {xxx['bannerId']:{'name':xxx['displayName'], 'contact':xxx['emailAddress'], 'is_primary':xxx['primaryIndicator']} for xxx in sec['faculty']},
        'meetings': [{'start':xxx['meetingTime']['beginTime'], 'end':xxx['meetingTime']['endTime'], 'building':xxx['meetingTime']['buildingDescription'], 'room':xxx['meetingTime']['room'], 'monday':xxx['meetingTime']['monday'], 'tuesday':xxx['meetingTime']['tuesday'], 'wednesday':xxx['meetingTime']['wednesday'], 'thursday':xxx['meetingTime']['thursday'], 'friday':xxx['meetingTime']['friday'], 'saturday':xxx['meetingTime']['saturday'], 'sunday':xxx['meetingTime']['sunday']} for xxx in sec['meetingsFaculty']],
        'campus': sec['campusDescription'],
        'delivery': sec['instructionalMethod']
    }

    for meeting in sec["meetingsFaculty"]:
        if meeting["meetingTime"] is not None:
            time = meeting["meetingTime"]
            if time["buildingDescription"] is not None and time["room"] is not None:

                rooms[time["buildingDescription"]].add(time["room"])
        

    cleaned_course_sections[sec['subject']+sec['courseNumber']].append(clean_sec)

    cleaned_sections[sec['courseReferenceNumber']] = clean_sec

In [192]:
cleaned_buildings = dict()

for bldg in rooms:
    cleaned_buildings[bldg] = {
        'rooms': list(rooms[bldg]) # cast the sets into lists so we can serialize to json. need to do this as sets are not hashable.
    }

In [193]:
# modify this to have lat longs for rooms once i get them. 
# dont really need the current lat long as it's just an avg and some of them aren't accurate..

for bldg in buildings:
    if cleaned_buildings.get(bldg):
        cleaned_buildings[bldg] = {
            'lat': buildings[bldg]['lat'],
            'long': buildings[bldg]['lon'],
            'rooms': cleaned_buildings.get(bldg)['rooms']
        }
    else:
        cleaned_buildings[bldg] = {
            'lat': buildings[bldg]['lat'],
            'long': buildings[bldg]['lon'],
            'rooms': []
        }

In [97]:
cleaned_courses = dict()
for course in courses.values():
    id = course['courseID']

    cleaned_courses[id] = {
        'pid': course['pid'],
        'description': course['description'],
        'notes': course['notes'],
        'link': f"https://www.uvic.ca/calendar/future/undergrad/index.php#/courses/{course['pid']}",
        'sections': cleaned_course_sections[id]
    }

In [116]:
cleaned_degrees = dict()

for degree in degrees.values():
    id = degree['code']
    if degree.get('requirements'):
        courses = re.findall(r'([A-Z]{2,4}\d{3}[A-Z]?\d?|[A-Z]{2,4})', degree.get('requirements'))
        potential_program_codes = [re.findall(r'[A-Z]{2,4}', code)[0] for code in courses]
        program_code = Counter(potential_program_codes).most_common(1)[0][0]
    
    cleaned_degrees[id] = {
        'code': id,
        'type': degree.get('cred'),
        'description': degree.get('description'), # parse this and make it useful. there are locally referenced links which won't work on other sites.
        'subject': program_code,
        'requirements': degree.get('requirements'), # parse this to extract a tree that holds logic for courses and requirements. will first need to get prereqs for each course. this will be a bit involved...
        'specializations': degree.get('specializations'),
        'notes': degree.get('notes'),
        'link': f"https://www.uvic.ca/calendar/future/undergrad/index.php#/programs/{degree.get('pid')}",
        
    }
    

In [195]:
print(f"{len(cleaned_degrees)} degrees")
print(f"{len(cleaned_courses)} courses")
print(f"{len(cleaned_course_sections)} courses in fall and spring 23/24")
print(f"{len(cleaned_courses)-len(cleaned_course_sections)} courses not being offered this year")
print(f"{len(cleaned_sections)} sections in fall and spring 23/24")
print()
print(f"{len(rooms)} buildings on campus. Rooms in use this year:")
for bldg in rooms:
    print(f"   {bldg} has {len(rooms[bldg])} rooms")

277 degrees
3547 courses
2726 courses in fall and spring 23/24
821 courses not being offered this year
7127 sections in fall and spring 23/24

27 buildings on campus. Rooms in use this year:
   David Strong Building has 13 rooms
   Clearihue Building has 45 rooms
   David Turpin Building has 11 rooms
   Elliott Building has 27 rooms
   Engineering Comp Science Bldg has 15 rooms
   Cornett Building has 24 rooms
   MacLaurin Building has 35 rooms
   Fine Arts Building has 6 rooms
   Hickman Building has 4 rooms
   McKinnon Building has 12 rooms
   Stadium has 1 rooms
   Field has 2 rooms
   Bob Wright Centre has 12 rooms
   Visual Arts Building has 13 rooms
   Human &amp; Social Development has 6 rooms
   Business &amp; Economics Building has 5 rooms
   Fraser Building has 9 rooms
   First Peoples House has 2 rooms
   Sŋ&eacute;qə ʔ&eacute;ʔləŋ | Sngequ House has 2 rooms
   Petch Building has 7 rooms
   Cunningham Building has 12 rooms
   Engineering Lab Wing has 21 rooms
   Jamie Cassel

In [197]:
# Not needed as all sections are in the courses.json
# with open("sections.json", "w") as f:
#     json.dump(cleaned_sections, f)

with open("output/courses.json", "w") as f:
    json.dump(cleaned_courses, f)

with open("output/degrees.json", "w") as f:
    json.dump(cleaned_degrees, f)

with open("output/buildings.json", "w") as f:
    json.dump(cleaned_buildings, f)